Goal is to develop machine learning code that takes input photon data from an array of my hodoscope and outputs where the beam impinged on the scintillating crystal.

Google Colab Installation of mlinphysics

In [3]:
COLAB_FOLDER = 'MLP' # change as needed
GITHUB_USER  = 'hbprosper'
GITHUB_REPO  = 'mlinphysics'
GITHUB_FOLDERS = ['mlinphysics']
#------------------------------------------------------
try:
    from google.colab import drive
    drive.mount('/content/gdrive')
    print('\nGoogle Drive mounted\n')
    IN_COLAB = True
except:
    print('\nRunning locally\n')
    IN_COLAB = False

if IN_COLAB:
    MYDRIVE     = '/content/gdrive/MyDrive'
    GITHUB_BASE = 'https://raw.githubusercontent.com'
    MAIN        = 'refs/heads/main'
    GITHUB_PATH = f'{MYDRIVE}/{COLAB_FOLDER}'
    #------------------------------------------------------
    %cd {GITHUB_PATH}
    %rm -f {GITHUB_PATH}/clone2colab.ipynb
    !wget -q {GITHUB_BASE}/{GITHUB_USER}/{GITHUB_REPO}/{MAIN}/clone2colab.ipynb
    %run {GITHUB_PATH}/clone2colab.ipynb

    %pip install torch_geometric


Running locally



Importing the required modules

In [4]:
# standard system modules
import os, sys

# standard module for tabular data
import pandas as pd

# standard module for array manipulation
import numpy as np

# standard module for high-quality plots
import matplotlib as mp
import matplotlib.pyplot as plt

# standard research-level machine learning toolkit from Meta (FKA: FaceBook)
import torch
import torch.nn as nn
import torch.utils.data as dt

# module to access data in Hieracrchical Data Format (HDF or H5 format)
import h5py

# module to plot pixelized images
import imageio.v3 as im

# module to reimport Python modules
import importlib

# module for saving/loading serialized Python objects
import joblib

# module for shell utilities
import shutil

# ML in physics module
import mlinphysics.nn as mlp
import mlinphysics.utils.data as dat
import mlinphysics.utils.monitor as mon





Computational Device

In [5]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'\n\tAvailable device: {str(DEVICE):4s}\n')


	Available device: cpu 



Loading the data sets

In [18]:
produced_photons_array = np.loadtxt('scan_CsI_Se82_master_produced_photon.dat')
absorbed_photons_array = np.loadtxt('scan_CsI_Se82_master_absorbed_photon.dat')
observed_photon_array_array = np.loadtxt('scan_CsI_Se82_master_observed_photon_array.dat')
observed_photon_total_array = np.loadtxt('scan_CsI_Se82_master_observed_photon_total.dat')
beam_position_array = np.loadtxt('scan_CsI_Se82_master_beam_position.dat')

#Normalizing each PMT of the array to the total number of photons seen by the PMTs
normalized_PMT_array = np.zeros((1296,16))

for i in range (1296):
    for m in range (16):
        normalized_PMT_array[i][m] = observed_photon_array_array[i][m] / observed_photon_total_array[i]

# print (normalized_PMT_array.shape)

print(beam_position_array)


[[-3.5 -3.5 -8.   0.   0.   1. ]
 [-3.5 -3.3 -8.   0.   0.   1. ]
 [-3.5 -3.1 -8.   0.   0.   1. ]
 ...
 [ 3.5  3.1 -8.   0.   0.   1. ]
 [ 3.5  3.3 -8.   0.   0.   1. ]
 [ 3.5  3.5 -8.   0.   0.   1. ]]


Separating the data sets into training, validating, and testing

In [25]:
# generates a random sequence of integers from 0-1295
randomized_indices = np.random.permutation(1296)

training_indices = randomized_indices[0:432]
validation_indices = randomized_indices[432:864]
evaluating_model_indicies = randomized_indices[864:1296]

# training, validation, and evaluation photon datasets
training_photon_arrays = np.zeros((432,16))
validation_photon_arrays = np.zeros((432,16))
evaluation_photon_arrays = np.zeros((432,16))

# training, validation, and evaluation position datasets
training_position_arrays = np.zeros((432,2))
validation_position_arrays = np.zeros((432,2))
evaluation_position_arrays = np.zeros((432,2))

for i in range (432):
    # filling the photon datasets
    training_photon_arrays[i][:] = normalized_PMT_array[training_indices[i]][:]
    validation_photon_arrays[i][:] = normalized_PMT_array[validation_indices[i]][:]
    evaluation_photon_arrays[i][:] = normalized_PMT_array[evaluating_model_indicies[i]][:]
    
    # filling the position datasets
    training_position_arrays[i][:] = beam_position_array[training_indices[i], 0:2]
    validation_position_arrays[i][:] = beam_position_array[validation_indices[i]][0:2]
    evaluation_position_arrays[i][:] = beam_position_array[evaluating_model_indicies[i]][0:2]

#reshaping the normalized data so that we have 432 images of 4x4 pixels
training_photons = training_photon_arrays.reshape(432, 4, 4)
validation_photons = validation_photon_arrays.reshape(432, 4, 4)
evaluating_photons = evaluation_photon_arrays.reshape(432, 4, 4)

print('training idices: ',training_indices[0])

print('training position', training_position_arrays[0][:])

print ('training photons', training_photon_arrays[0][:])
###############################################

# REMEMBER THAT IF YOU WANT TO COMPARE THE INDEX THAT YOU GET WITH THE DATA SET,
# YOU MUST ADD +1 TO THE INDEX VALUE. THE POSSIBLE NUMBERS RANGE FROM 0 TO 1296-1 (1295)
# WHILE THE DATA SET STARTS FROM 1

###############################################


training idices:  649
training position [ 0.1 -3.3]
training photons [0.05166662 0.04860686 0.04795064 0.05025786 0.05860697 0.05616966
 0.05514111 0.05710189 0.06649824 0.06565226 0.06541435 0.06591317
 0.07671038 0.07905066 0.07794018 0.07731917]


Configuration of the model

According to this stackexchange conversation

https://stackoverflow.com/questions/4752626/epoch-vs-iteration-when-training-neural-networks

one epoch = one forward pass and one backward pass of all the training examples

batch size = the number of training examples in one forward/backward pass. The higher the batch size, the more memory space you'll need

number of iterations = number of passes, each pass using [batch size] number of examples. 
    To be clear, one pass = one forward pass + one backward pass (forward and backward passes are not counted as two different passes)

For example: if you have 1000 training examples, and your batch size is 500, then it will take 2 iterations to complete 1 epoch.


In [ ]:

model_name = 'impinging_position'

# choose whether to create or load a configuration file
load_existing_config = False

if load_existing_config:
    config = mlp.Config(f'{model_name}.yaml')
else:
    # create new config
    config = mlp.Config(model_name)

    n_images = 1296
    batch_size = 18
    n_iters_per_epoch = 24 # number of iterations per epoch
    train_size = n_iters_per_epoch * batch_size
    test_size = 432

    val_size = n_images - train_size - test_size

    config('batch_size', batch_size)
    config('train_size', train_size)
    config('test_size', test_size)
    config('val_size', val_size)

    config('monitor_step', 27) # set monitor training every n
    config('delete', True) # delete losses file before training, if True
    config('frac', 0.015) # save model if average loss decreases by more than frac percent

    config('n_epochs', 200)
    config('n_iters_per_epoch', n_iters_per_epoch)
    config('n_iterations', config('n_epochs') * config('n_iters_per_epoch'))

    config('n_steps', 4) # number of training steps
    config('n_iters_per_step', config('n_iterations') // config('n_steps'))
    
    config('base_lr', 1.e-3) # initial learning rate
    config('gamma', 0.8) # learning rate scale factor

    print(f'\nSave configuration to file {config.cfg_filename}\n')
    
    config.save()

print(config)



Save configuration to file runs/2025-11-19_2100/impinging_position_config.yaml

